In [50]:
import pandas as pd
import os


In [51]:
prop  = pd.read_csv('../data/curated/clean_data.csv')
len(prop)

9219

In [52]:
#Drop the data that has the price less than $10
prop.drop(prop[prop['Prices'] < 10].index, inplace = True)
prop = prop.reset_index(drop=True)
len(prop)

8794

Create a Suburb column in the property dataframe, using the word after \n in the Address column


In [53]:
prop['Suburb'] = ""

for i in range(len(prop['Suburb'])):
     prop['Suburb'][i] = prop['Address'][i].split('\n', 1)[-1].split(' VIC')[0]


###############################################


addresses = prop['Address']
suburbs_prop = []
for address in addresses:
     suburb = address.split('\n', 1)[-1].split(' VIC')[0]
     if suburb in suburbs_prop:
          continue
     else:
          suburbs_prop.append(suburb)

len(prop)

/var/folders/mc/_4clq8l155jdkpr2knhw_mw00000gn/T/ipykernel_8265/1749859943.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prop['Suburb'][i] = prop['Address'][i].split('\n', 1)[-1].split(' VIC')[0]


8794

In [54]:
#Find the SA2 area name for each row in the ABS data set

abs = pd.read_csv('../data/curated/2021ABS_csv')
for i in range(len(abs['SA2 Area Name'])):
    abs['SA2 Area Name'][i] = abs['SA2 Area Name'][i].split('summary_ ', 1)[-1].split(' SA2', 1)[0].upper()
    
    if '-' in abs['SA2 Area Name'][i]:

        abs['SA2 Area Name'][i] = abs['SA2 Area Name'][i].replace(' -', '')


abs_sa2 =[]
for i in range(len(abs['SA2 Area Name'])):
    abs_sa2.append(abs['SA2 Area Name'][i].upper())

/var/folders/mc/_4clq8l155jdkpr2knhw_mw00000gn/T/ipykernel_8265/2508610607.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abs['SA2 Area Name'][i] = abs['SA2 Area Name'][i].split('summary_ ', 1)[-1].split(' SA2', 1)[0].upper()
/var/folders/mc/_4clq8l155jdkpr2knhw_mw00000gn/T/ipykernel_8265/2508610607.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abs['SA2 Area Name'][i] = abs['SA2 Area Name'][i].replace(' -', '')


In [55]:
external_features = [
    'Estimated resident population (no.)',
    'Population density (persons/km2)',
    'Median age - persons (years)',
    'Working age population (aged 15-64 years) (no.)',
    'Employed (no.)', 'Unemployed (no.)', 'Renter (no.)',
    'Health care and social assistance (no.)','Financial and insurance services (no.)',
    'Administrative and support services (no.)','Education and training (no.)',
    'Accommodation and food services (no.)','Public administration and safety (no.)',
    'Value of residential building ($m)', 'Houses - total (no.)', 
    'Townhouses - total (no.)', 'Apartments - total (no.)',
    'Median weekly household rental payment ($)', 'Rented (no.)',
    'Count of homeless persons (no.)','Used at least one form of public transport (train, tram, bus, ferry) (no.)'
]

#Join the two data sets by name, so iof they have the same suburb give that property the corresponding data
external  = prop[['Address', 'Suburb', 'Prices']]
for feature in external_features:
    external[feature] = ''

for i in range(len(external['Suburb'])):
    if external['Suburb'][i] in abs_sa2:
        row_number = abs.index.get_loc(abs[abs['SA2 Area Name'] == external['Suburb'][i]].index[0])
        for j in external_features:
            external[j][i] = abs[j][row_number]

/var/folders/mc/_4clq8l155jdkpr2knhw_mw00000gn/T/ipykernel_8265/1421891289.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  external[feature] = ''
/var/folders/mc/_4clq8l155jdkpr2knhw_mw00000gn/T/ipykernel_8265/1421891289.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  external[feature] = ''
/var/folders/mc/_4clq8l155jdkpr2knhw_mw00000gn/T/ipykernel_8265/1421891289.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [56]:
len(external)

8794

In [57]:
#area codes to drop so we can get more rows filled
drop_words = ['NORTH', 'SOUTH', 'EAST', 'WEST', '(VIC.)']

#This will make Keilor East and Keilor West both just Keilor
for i in range(len(abs['SA2 Area Name'])):
    for word in drop_words:
        if word in abs['SA2 Area Name'][i]:
            word = ' '+word
            abs['SA2 Area Name'][i] = abs['SA2 Area Name'][i].replace(word, '')


/var/folders/mc/_4clq8l155jdkpr2knhw_mw00000gn/T/ipykernel_8265/2061638818.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abs['SA2 Area Name'][i] = abs['SA2 Area Name'][i].replace(word, '')


In [58]:
#If the name of the suburb is the same add the rows together
sum_dict = {}
for column in external_features:
    sum_dict[column] = 'sum'

abs_grouped = abs.groupby('SA2 Area Name').agg(sum_dict).reset_index()

In [59]:
#Now apply the added rows to the data
abs_sa2_grouped =[]
for i in range(len(abs_grouped['SA2 Area Name'])):
    abs_sa2_grouped.append(abs_grouped['SA2 Area Name'][i].upper())

for i in range(len(external['Suburb'])):
    if external['Suburb'][i] in abs_sa2_grouped:
        row_number = abs_grouped.index.get_loc(abs_grouped[abs_grouped['SA2 Area Name'] == external['Suburb'][i]].index[0])
        for j in external_features:
            external[j][i] = abs_grouped[j][row_number]

/var/folders/mc/_4clq8l155jdkpr2knhw_mw00000gn/T/ipykernel_8265/10110642.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  external[j][i] = abs_grouped[j][row_number]


In [60]:
external.drop(external[external['Estimated resident population (no.)'] == ''].index, inplace = True)
external = external.reset_index(drop=True)

In [61]:
len(external)

5040

In [64]:
external_na = external.dropna(axis = 0)
external_na = external_na.reset_index(drop=True)
len(external_na)

4371

In [63]:
external_na.to_csv('../data/curated/Property_external.csv', index_label=False)